# 作业5

### 第1题

利用 PySpark 实现一个分布式估计圆周率 $\pi$ 的程序，原理如下：

在正方形 $\{(x,y):-1\le x \le 1, -1\le y \le 1\}$ 中随机生成 $N$ 个独立的均匀分布随机数 $(X_i,Y_i)$，其中每个点 $(X_i,Y_i)$ 落入圆 $R=\{(x,y): x^2+y^2\le 1\}$ 的概率是 $\pi/4$。因此，如果随机生成的 $N$ 个点中有 $n$ 个落入圆 $R$ 中，那么 $\pi$ 的估计就是 $4n/N$。

![](https://media.geeksforgeeks.org/wp-content/uploads/MonteCarlo.png)

现在我们采用分布式的方法并行模拟大量的随机数。考虑将所有的点分成100组，每组生成10000个点，每组独立产生随机数并计算落入圆内的数量，最后将所有100组的结果汇总并得出最终 $\pi$ 的估计。为了使结果可重复，第 $i$ 组在生成随机数时使用 $i$ 作为随机数种子。PySpark 使用本地模式，开启 8 个 CPU 核心。

**提示**：使用标准方法启动 PySpark 后，可以利用 `sc.parallelize()` 从一个迭代器或列表生成 RDD，如 `sc.parallelize(range(10))` 和 `sc.parallelize([1, 2, 3])`。

In [11]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
# 单机模式
spark = SparkSession.builder.master("local[8]").appName("Reading Text").getOrCreate()
sc = spark.sparkContext
# sc.setLogLevel("ERROR")
print(spark)
print(sc)

<SparkContext master=local[8] appName=Reading Text>


In [6]:
import numpy as np
m = 100
p = 10000

def square(i):
    global p
    np.random.seed(i+1)
    x = np.random.uniform(low=-1, high=1, size=p)
    y = np.random.uniform(low=-1, high=1, size=p)
    num = np.sum(np.square(x) + np.square(y) <= 1)
    return num, p

rdd = sc.parallelize(range(m))
n, N = rdd.map(square).reduce(lambda x, y: (x[0] + y[0], x[1] + y[1]))
pi_hat = 4 * n / N
pi_hat

3.142256

### 第2题

在 `lec12-admm-lasso.ipynb` 的基础上，利用 ADMM 算法求解 Lasso 问题

$$\frac{1}{2}\Vert y-X\beta\Vert^2+\lambda \Vert \beta\Vert_1,$$

并将其封装成一个函数：

```python
admm_lasso(X, y, lam, rho=1.0, maxit=10000, eps=1e-3, verbose=0)
```

1. 其中 `X` 是 $n\times p$ 的自变量矩阵，`y` 是 $n\times 1$ 的因变量向量，`lam` 是惩罚项参数 $\lambda$，`rho` 是 ADMM 算法的 $\rho$ 参数，`maxit` 是最大迭代次数，`eps` 是 ADMM 收敛的残差临界值，`verbose` 表示是否输出迭代信息，如果 $>0$，则每隔 1000 次迭代打印出当前的两类残差，如果 $\le 0$ 否则不输出任何信息。
2. 参考 `lec12-admm-lad.ipynb` 中的 Cholesky 分解方法，只对矩阵进行一次分解，从而在每次迭代中高效地求解线性方程组。
3. 函数需返回两个量，第一个表示实际使用的迭代次数，第二个表示估计的回归系数。

In [1]:
from scipy.linalg import cho_factor, cho_solve

def soft_thresholding(a, k):
    return np.sign(a) * np.maximum(0.0, np.abs(a) - k)

def admm_lasso(X, y, lam, rho=1.0, maxit=10000, eps=1e-3, verbose=0):
    p = X.shape[1]
    z = np.zeros(p)
    u = np.zeros(p)
    # Cholesky 分解
    c, lower = cho_factor(X.T.dot(X) + rho * np.eye(p))

    for i in range(maxit):
        # x 更新
        xnew = cho_solve((c, lower), X.T.dot(y)+rho*(z - u))
        # z 更新
        znew = soft_thresholding(xnew + u, lam / rho)
        # u 更新
        unew = u + xnew - znew
        # 计算残差大小
        resid_r_norm = np.linalg.norm(unew - u)
        resid_s_norm = rho * np.linalg.norm(znew - z)
        # 更新 x、z 和 u 的取值
        z = znew
        u = unew
        # 打印残差信息，判断是否收敛
        if verbose > 0 and i % 1000 == 0:
            print(f"Iteration {i}, ||r|| = {resid_r_norm:.6f}, ||s|| = {resid_s_norm:.6f}")
        if resid_r_norm <= eps and resid_s_norm <= eps:
            break
    return i+1, z

利用模拟训练集数据测试上述编写的函数：

In [3]:
import numpy as np
np.random.seed(123)
n = 1000
p = 30
nz = 20
Xtrain = np.random.normal(size=(n, p))
# 真实的 x 只有前20个元素非零，其余均为0
beta = np.random.normal(size=nz)
beta = np.concatenate((beta, np.zeros(p - nz)))
ytrain = Xtrain.dot(beta) + np.random.normal(size=n)
beta

array([-1.05417044, -0.78301134,  1.82790084,  1.7468072 ,  1.3282585 ,
       -0.43277314, -0.6686141 , -0.47208845,  1.05554064,  0.67905585,
        0.14814832,  1.04294573,  0.28718991,  1.55577283,  0.97031604,
        0.39737593,  1.15394013, -0.00333042,  1.30948521, -0.90230241,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

In [4]:
admm_lasso(Xtrain, ytrain, lam=0.1 * n, maxit=10000, eps=1e-3, verbose=1)

Iteration 0, ||r|| = 4.571809, ||s|| = 0.000000
Iteration 1000, ||r|| = 0.067950, ||s|| = 0.000009
Iteration 2000, ||r|| = 0.019016, ||s|| = 0.000003
Iteration 3000, ||r|| = 0.007483, ||s|| = 0.000001
Iteration 4000, ||r|| = 0.002978, ||s|| = 0.000000
Iteration 5000, ||r|| = 0.001199, ||s|| = 0.000000


(5202,
 array([-0.98844619, -0.72995199,  1.72843395,  1.66188615,  1.18779108,
        -0.1944663 , -0.59471119, -0.39143086,  1.01063023,  0.57378667,
         0.03363641,  0.93113597,  0.22189703,  1.51032137,  0.90777987,
         0.29344991,  1.08151311, -0.        ,  1.17431918, -0.78857287,
        -0.        ,  0.        , -0.        , -0.        , -0.        ,
        -0.        , -0.        , -0.        ,  0.        , -0.        ]))

In [5]:
admm_lasso(Xtrain, ytrain, lam=0.01 * n, maxit=10000, eps=1e-3, verbose=0)

(1287,
 array([-1.07555904, -0.81446022,  1.79118556,  1.72909346,  1.27448621,
        -0.30689747, -0.66946929, -0.4730217 ,  1.09124222,  0.66934076,
         0.12487601,  1.02527211,  0.30210648,  1.58722372,  0.96866322,
         0.38493746,  1.15919477, -0.03766987,  1.27397237, -0.90126783,
        -0.0074241 ,  0.01919108, -0.06064217, -0.02572023, -0.01930401,
        -0.03339472,  0.        , -0.01529796,  0.02225086, -0.02012609]))

### 第3题

利用第2题中编写的函数，对一个新的测试集数据做预测。首先生成模拟数据：

In [6]:
np.random.seed(123)
ntest = 500
p = 30
Xtest = np.random.normal(size=(ntest, p))
ytest = Xtest.dot(beta) + np.random.normal(size=ntest)

取 $\lambda=0.1 n$，利用训练集估计回归系数，然后对测试集的因变量做预测，计算预测结果的均方误差，即
$$
MSE=\frac{1}{n_{test}}\sum_{i=1}^{n_{test}}(\hat{y}_i-y_i)^2,
$$
其中 $y_i$ 是第 $i$ 个测试集观测的因变量取值，$\hat{y}_i=x_i'\hat{\beta}$ 是第 $i$ 个观测的因变量预测值。

In [7]:
iter_num, beta_hat = admm_lasso(Xtrain, ytrain, lam=0.1 * n, maxit=10000, eps=1e-3, verbose=0)
y_hat = Xtest.dot(beta_hat)
MSE = np.mean(np.square(y_hat - ytest))
MSE

1.1783546761616057

利用 PySpark 来并行地对 Lasso 模型的 $\lambda$ 参数进行调优，并考察 $\rho$ 参数对算法收敛速度的影响。取 $\rho=0.1,0.2,\ldots,1.0$，$\lambda=0.1n,0.01n,0.001n$。对于 $\rho$ 和 $\lambda$ 的这 30 个组合，分别利用训练集拟合 Lasso 模型，返回迭代次数，并计算在测试集上的预测 MSE。最终输出如下的结果：

```
rho = 0.1, lambda/n = 0.1, niter = ..., mse = ...
rho = 0.1, lambda/n = 0.01, niter = ..., mse = ...
...
```

**提示**：先生成 $\rho$ 和 $\lambda$ 所有组合的列表，类似于 `params = [(0.1, 0.1), (0.1, 0.01), (0.1, 0.001), (0.2, 0.1), ...]`，然后利用 `sc.parallelize(params)` 生成一个 RDD，最后对这个 RDD 进行 `map()` 和 `collect()` 操作。

In [20]:
params = [(rho, lam) for rho in np.linspace(0.1, 1.0, num=10) for lam in [0.1, 0.01, 0.001]]

rdd = sc.parallelize(params)

def parmas_fit(param, Xtrain, ytrain, Xtest, ytest):
    rho, lam = param
    global n
    niter, beta_hat = admm_lasso(Xtrain, ytrain, lam*n, rho, maxit=10000, eps=1e-3, verbose=0)
    y_hat = Xtest.dot(beta_hat)
    mse = np.mean(np.square(y_hat - ytest))
    return niter, mse

results = rdd.map(lambda p: parmas_fit(p, Xtrain, ytrain, Xtest, ytest)).collect()

for (rho, lambda_n), (niter, mse) in zip(params, results):
    print(f"rho = {rho:.1f}, lambda/n = {lambda_n}, niter = {niter}, mse = {mse}") 

rho = 0.1, lambda/n = 0.1, niter = 10000, mse = 1.1862413289385212
rho = 0.1, lambda/n = 0.01, niter = 10000, mse = 1.0462119621406527
rho = 0.1, lambda/n = 0.001, niter = 2469, mse = 1.054495641790964
rho = 0.2, lambda/n = 0.1, niter = 10000, mse = 1.1784220082446604
rho = 0.2, lambda/n = 0.01, niter = 6429, mse = 1.0462129060335947
rho = 0.2, lambda/n = 0.001, niter = 1235, mse = 1.0544962286867647
rho = 0.3, lambda/n = 0.1, niter = 10000, mse = 1.1783872213386157
rho = 0.3, lambda/n = 0.01, niter = 4286, mse = 1.0462129058026297
rho = 0.3, lambda/n = 0.001, niter = 824, mse = 1.0544803449550018
rho = 0.4, lambda/n = 0.1, niter = 10000, mse = 1.1783671606734305
rho = 0.4, lambda/n = 0.01, niter = 3215, mse = 1.0462129061369436
rho = 0.4, lambda/n = 0.001, niter = 619, mse = 1.0544803244140812
rho = 0.5, lambda/n = 0.1, niter = 10000, mse = 1.1783561563633667
rho = 0.5, lambda/n = 0.01, niter = 2572, mse = 1.0462129059005953
rho = 0.5, lambda/n = 0.001, niter = 496, mse = 1.0544803174